In [3]:
using Markdown
using InteractiveUtils
using BenchmarkTools

# FixArgs.jl: Reusing meaningful names in structural expressions
## Gustavo Nunes Goretkin
### JuliaCon 2021
- discuss functions used for their name and not for invocation
- expressions encoded "structurally" in types
- experimental package (FixArgs.jl) to showcase this pattern
- focus on the idea rather than the specific implementation

To concatenate two `AbstractVector`s, use `vcat`:

In [4]:
vcat(1:5, 1:5)

10-element Vector{Int64}:
 1
 2
 3
 4
 5
 1
 2
 3
 4
 5

Suppose we have a `Vector` of `Vector`s, and we want to concatenate all of the inner `Vector`s into one `Vector`.

In [5]:
vs = [collect(1:10) for _ = 1:2000]; # `Vector` of `<:AbstractVector`s,  really.

Apply binary operation over a sequence using `reduce`:

In [6]:
reduce(vcat, vs) |> string

"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 

Now let us do essentially the same computation, but instead of directly using `vcat`, we define a function that just calls `vcat`:

```julia
reduce((_1, _2) -> vcat(_1, _2), vs)
```

And let's time the two.

In [7]:
@benchmark reduce(vcat, vs)

BenchmarkTools.Trial: 
  memory estimate:  172.08 KiB
  allocs estimate:  3
  --------------
  minimum time:     44.748 μs (0.00% GC)
  median time:      225.423 μs (0.00% GC)
  mean time:        271.578 μs (13.33% GC)
  maximum time:     19.636 ms (98.35% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [8]:
@benchmark reduce((_1, _2) -> vcat(_1, _2), vs)

BenchmarkTools.Trial: 
  memory estimate:  76.74 MiB
  allocs estimate:  3592
  --------------
  minimum time:     105.503 ms (10.97% GC)
  median time:      125.224 ms (20.95% GC)
  mean time:        132.551 ms (19.43% GC)
  maximum time:     263.981 ms (17.58% GC)
  --------------
  samples:          39
  evals/sample:     1

It is ~500× slower in this case.

Why? It is not because anonymous functions are slow (they aren't). Totally different code is running each case. (e.g. allocate output all at once)

In [9]:
methods(reduce)

# 9 methods for generic function "reduce":
[1] reduce(op, a::Number) in Base at reduce.jl:458
[2] reduce(::typeof(merge), items::Vector{var"#s79"} where var"#s79"<:Dict) in Base at dict.jl:711
[3] reduce(::typeof(vcat), S::SharedArrays.SharedVector{T} where T) in SharedArrays at /Applications/Julia-1.6.app/Contents/Resources/julia/share/julia/stdlib/v1.6/SharedArrays/src/SharedArrays.jl:572
[4] reduce(::typeof(vcat), A::AbstractVector{var"#s79"} where var"#s79"<:(AbstractVecOrMat{T} where T)) in Base at abstractarray.jl:1569
[5] reduce(::typeof(hcat), S::SharedArrays.SharedVector{T} where T) in SharedArrays at /Applications/Julia-1.6.app/Contents/Resources/julia/share/julia/stdlib/v1.6/SharedArrays/src/SharedArrays.jl:573
[6] reduce(::typeof(hcat), A::AbstractVector{var"#s79"} where var"#s79"<:(AbstractVecOrMat{T} where T)) in Base at abstractarray.jl:1572
[7] reduce(f, S::SharedArrays.SharedArray) in SharedArrays at /Applications/Julia-1.6.app/Contents/Resources/julia/share/julia/stdlib/v1.6/SharedArrays/src/SharedArrays.jl:569
[8] reduce(op, A::AbstractArray; kw...) in Base at reducedim.jl:359
[9] reduce(op, itr; kw...) in Base at reduce.jl:456

## Spelling
There is a reason that "spelling bees" were invented for English-language writing system.

Two key features:
1. multiple dispatch
2. the function is encoded in the type domain (i.e. each function is its own type)

enables defining special case for `reduce(vcat, ...)`.

Otherwise, would need name e.g. `reduce_vcat(...)`, or perhaps `flatten(...)`.

Personally, the first spelling is better. It combines existing and meaningful names instead of introducing a new ad-hoc name.

Calling `reduce(vcat, ...)` might not call `vcat`. `vcat` is used as a _name_.

More so than in other ecosystems, the Julia community aims to determine a generic meaning for a given function name. This is difficult and fundamental design work. It enables generic programming. It is good to reuse these names!

There are multiple converging motivations for the idea in this talk:

* Get extra value from careful function name / meaning pairs

* Generalize `Base.Fix1`/`Base.Fix2`

* Symbolic Computation / Lazy Computation

* Explore a combination of structural and of nominal types

## Demo of `Base.Fix2`
`==` is a two-argument function. `==(a, b)` is identical to `a == b`

In [10]:
f1 = ==(50)

(::Base.Fix2{typeof(==), Int64}) (generic function with 1 method)

In [11]:
f2 = x -> x == 50

#7 (generic function with 1 method)

`f1` and `f2` compute the same function. Some might say that they are different names for the same function.



And in this case, names matter!

In [12]:
findfirst(f1, -1000:1000) # O(1)

1051

In [13]:
findfirst(f2, -1000:1000) # O(length(range))

1051

In [14]:
@which findfirst(f1, -1000:1000)

findfirst(p::Union{Base.Fix2{typeof(==), T}, Base.Fix2{typeof(isequal), T}}, r::AbstractUnitRange) where T<:Integer in Base at array.jl:1912

```julia
findfirst(p::Union{Fix2{typeof(isequal),T},Fix2{typeof(==),T}}, r::AbstractUnitRange) where {T<:Integer} =
    first(r) <= p.x <= last(r) ? 1+Int(p.x - first(r)) : nothing
```

This is why I love Julia. Imagine a plotting library that supports unevenly-spaced axis ticks.

Code written in terms of `findfirst` can support unevenly-spaced ticks, and still be (runtime) efficient when using evenly-spaced ticks.

The same code can handle a general case, without affecting performance in the special case!

The types `Fix1`/`Fix2` fix one argument of a two-argument function.


`FixArgs.jl` provides a generalization of `Fix1` and `Fix2` in a few ways:
1. A function of any positional arity can be used, and any number of its arguments can be bound, allowing the remaining arguments to be provided later.
2. A function can have its keyword arguments bound.
3. The function `x -> f(x, b)` is represented with types:
   - a [`Lambda`](https://goretkin.github.io/FixArgs.jl/dev/#FixArgs.Lambda) to represent function (`args -> body`)
   - a [`Call`](https://goretkin.github.io/FixArgs.jl/dev/#FixArgs.Call) to represent the function *call* (`f(...)`) in the body
   - a [`ArgPos`](https://goretkin.github.io/FixArgs.jl/dev/#FixArgs.ArgPos) to represent the `x` in the body of the lambda function

## A quick fix

Partial application with keyword arguments

```julia
"""
    isapprox(x; kwargs...) / ≈(x; kwargs...)
Create a function that compares its argument to `x` using `≈`, i.e. a function equivalent to `y -> y ≈ x`.
The keyword arguments supported here are the same as those in the 2-argument `isapprox`.
"""
isapprox(y; kwargs...) = x -> isapprox(x, y; kwargs...)
```

All other docstrings in `Base` with "Create a function" use `Base.Fix2`, but here an anonymous function is used.

## Illustration
Do we want a way to encode lambda calculus in types?

Would it ever be useful to fix all of the arguments of a function?

Consider the `/` (division) function.

If you fix its two arguments, that looks a lot like `Rational`.

In [15]:
using FixArgs
half = @xquote 1 / 2

Call(Some(/), FrankenTuple((Some(1), Some(2)), NamedTuple()))

A macro helps create `Lambda` / `Call` / `ArgPos`, etc. objects

In [16]:
half * half

LoadError: MethodError: no method matching *(::FixArgs.Call{Some{typeof(/)}, FrankenTuples.FrankenTuple{Tuple{Some{Int64}, Some{Int64}}, (), Tuple{}}}, ::FixArgs.Call{Some{typeof(/)}, FrankenTuples.FrankenTuple{Tuple{Some{Int64}, Some{Int64}}, (), Tuple{}}})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560

Another macro helps define the types

In [17]:
function Base.:*(a::(@xquoteT ::S / ::S), b::(@xquoteT ::S / ::S)) where S
    (n1, d1) = something.(Tuple(a.args))
    (n2, d2) = something.(Tuple(b.args))
    @xquote $(n1 * n2) / $(d1 * d2)
end

In [18]:
half * half

Call(Some(/), FrankenTuple((Some(1), Some(4)), NamedTuple()))

Where does the connection between `Rational` and `/` happen in `Base`? [Here](https://github.com/JuliaLang/julia/blob/51f57406038df9f438edf911c8d1b59c9c1af4b1/base/rational.jl#L112).

    
```julia
AbstractFloat(x::Rational) = (float(x.num)/float(x.den))::AbstractFloat
```

In [19]:
xeval(half) # evaluate the lazy representation

0.5

## memory and code generation comparison

In [20]:
sizeof(half)

16

In [21]:
sizeof(1 // 2)

16

In [22]:
reinterpret(Int, [half])

2-element reinterpret(Int64, ::Vector{FixArgs.Call{Some{typeof(/)}, FrankenTuples.FrankenTuple{Tuple{Some{Int64}, Some{Int64}}, (), Tuple{}}}}):
 1
 2

In [23]:
@code_native xeval(half)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ eval.jl:79 within `xeval' @ eval.jl:44
; │┌ @ eval.jl:104 within `_xapply'
; ││┌ @ FrankenTuples.jl:405 within `ftcall'
; │││┌ @ int.jl:93 within `/'
; ││││┌ @ float.jl:206 within `float'
; │││││┌ @ float.jl:191 within `AbstractFloat'
; ││││││┌ @ float.jl:94 within `Float64'
	vcvtsi2sdq	(%rdi), %xmm0, %xmm0
	vcvtsi2sdq	8(%rdi), %xmm1, %xmm1
; ││││└└└
; ││││ @ int.jl:93 within `/' @ float.jl:335
	vdivsd	%xmm1, %xmm0, %xmm0
; │└└└
; │ @ eval.jl:79 within `xeval'
	retq
; └


In [24]:
@code_native float(1//2)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ float.jl:206 within `float'
; │┌ @ rational.jl:112 within `AbstractFloat'
; ││┌ @ float.jl:206 within `float'
; │││┌ @ float.jl:191 within `AbstractFloat'
; ││││┌ @ float.jl:94 within `Float64'
	vcvtsi2sdq	(%rdi), %xmm0, %xmm0
	vcvtsi2sdq	8(%rdi), %xmm1, %xmm1
; ││└└└
; ││┌ @ float.jl:335 within `/'
	vdivsd	%xmm1, %xmm0, %xmm0
; │└└
	retq
; └


This is not a drop-in replacement for `Rational`, however, because

In [25]:
1//2 isa Number

true

In [26]:
(@xquote 1 / 2) isa Number

false

## Different than `Expr`
`/` in `:(/(x, y))` is not `Base.:/`, but just `:/`

In [27]:
dump(:(x / y))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol /
    2: Symbol x
    3: Symbol y


In [28]:
let x = 1, y =2
    @xquote x / y
end

Call(Some(/), FrankenTuple((Some(1), Some(2)), NamedTuple()))

## Why?
Using this instead of `Base.Rational` seems pretty silly and confusing. Possible benefits:

* Some users want a "rational" type where the numerator and the denominator are not constrained to be the same type.

* a fixed-point number is one of these rational types where the denominator is "static" (a singleton type such that the numerical value is encoded in the type domain). Check the documentation for `FixArgs.jl` examples!

In [29]:
MyQ0f7(x) = (@xquote $(Int8(x)) / 128::::S) # mark an argument as "static"
MyQ0f7(3)

Call(Some(/), FrankenTuple((Some(3), Val{128}()), NamedTuple()))

### A combination of structural and nominal typing
Avoid the requirement to choose names like `Rational`, `num`, `den`.
The arguments can be distinguished by the role they play with respect to the function. Only the function needs a name.

In [30]:
@xquote x -> x # `identity` could be an alias for this

Lambda(FixArgs.Arity{1, Nothing}(), arg_pos(1, 0))

In [31]:
x -> x

#9 (generic function with 1 method)

In [32]:
x -> x # gets a different name, even though structurally identical.

#11 (generic function with 1 method)

In [33]:
@xquote x -> x((y -> x))

FixNew(FixArgs.Arity{1, Nothing}(),arg_pos(1, 0),FrankenTuple((Lambda(FixArgs.Arity{1, Nothing}(), arg_pos(1, 1)),), NamedTuple()))

## Existing patterns
Any "eager" function now has a lazy representation.

`Base.Generator` could be a lazy representation of `map(f, itr)`

```julia
Base.Iterators.Filter(flt, itr)
```

could be replaced with

```julia
@xquote filter(flt, itr)
```

The type `Base.Iterators.Filter` could be an alias for

```julia
(@xquoteT filter(::F, ::I)) where {F, I}
```
to keep the existing symbolic "rules" such as [this one](https://github.com/JuliaLang/julia/blob/ef14131db321f8f5a815dd05a5385b5b27d87d8f/base/iterators.jl#L463):

```julia
reverse(f::Filter) = Filter(f.flt, reverse(f.itr))
```

[`Base.Iterators.Flatten(iterator_of_iterators)`](https://github.com/JuliaLang/julia/blob/ef14131db321f8f5a815dd05a5385b5b27d87d8f/base/iterators.jl#L1038-L1040) can be represented as `@xquote reduce(vcat, iterator_of_iterators)`

### `Base.literal_pow` 

In [34]:
Meta.@lower x^2

:($(Expr(:thunk, CodeInfo(
    @ none within `top-level scope'
1 ─ %1 = Core.apply_type(Base.Val, 2)
│   %2 = (%1)()
│   %3 = Base.literal_pow(^, x, %2)
└──      return %3
))))

Could instead lower to approximately
```
%1 = @xquote x^(2::::S) # i.e. something roughly like `Call(^, x, Val(2))`
%2 = xeval(%1)
```

### Broadcasting
[Call](https://github.com/JuliaLang/julia/blob/d06c2a97be3f643d403c4069955e135823ff9fd0/base/broadcast.jl#L152-L173)

```julia
struct Broadcasted{Style<:Union{Nothing,BroadcastStyle}, Axes, F, Args<:Tuple} <: Base.AbstractBroadcasted
    f::F
    args::Args
    axes::Axes
end
```

(extra information `Style` and `axes`)

[xeval](https://github.com/JuliaLang/julia/blob/1910a7685a86d44041a98ff874f92e480fc44632/base/broadcast.jl#L904)

```julia
@inline materialize(bc::Broadcasted) = copy(instantiate(bc))
# [... `copyto!` methods ...]
```

[LazyArrays.jl](https://github.com/JuliaArrays/LazyArrays.jl)

[Call](https://github.com/JuliaArrays/LazyArrays.jl/blob/6b5cee2e8ed355a8e7590c3ae860faed4e816e19/src/lazyapplying.jl#L184-L187)

```julia
struct ApplyArray{T, N, F, Args<:Tuple} <: LazyArray{T,N}
    f::F
    args::Args
end

# [...]

const Vcat{T,N,I<:Tuple} = ApplyArray{T,N,typeof(vcat),I}

```

[xeval](https://github.com/JuliaArrays/LazyArrays.jl/blob/dff5924cd8b52c62a34cce16372381bb8a9e35cb/src/lazyconcat.jl#L20-L27)
```julia
function instantiate(A::Applied{DefaultApplyStyle,typeof(vcat)})
    isempty(A.args) && return A
    m = size(A.args[1],2)
    for k=2:length(A.args)
        size(A.args[k],2) == m || throw(ArgumentError("number of columns of each array must match (got $(map(x->size(x,2), A)))"))
    end
    Applied{DefaultApplyStyle}(A.f,map(instantiate,A.args))
end
```


[LazyStacks.jl](https://github.com/mcabbott/LazyStack.jl)

[Call](https://github.com/mcabbott/LazyStack.jl/blob/a02740adae65cd27acc0257a61a482fdebf2e2eb/src/LazyStack.jl#L82-L84)
```julia
struct Stacked{T,N,AT} <: AbstractArray{T,N}
    slices::AT
end
```


[xeval](https://github.com/mcabbott/LazyStack.jl/blob/a02740adae65cd27acc0257a61a482fdebf2e2eb/src/LazyStack.jl#L111)
```julia
Base.collect(x::Stacked{T,2,<:AbstractArray{<:AbstractArray{T,1}}}) where {T} = reduce(hcat, x.slices)
```

[LazySets.jl](https://github.com/JuliaReach/LazySets.jl)

[Call](https://github.com/JuliaReach/LazySets.jl/blob/4e3dc1668a0265f748fef4a5ced93c97337c2623/src/LazyOperations/Intersection.jl#L98-L108)

```julia
struct Intersection{N, S1<:LazySet{N}, S2<:LazySet{N}} <: LazySet{N}
    X::S1
    Y::S2
    cache::IntersectionCache
    # [...]
end
```

[xeval](https://github.com/JuliaReach/LazySets.jl/blob/4e3dc1668a0265f748fef4a5ced93c97337c2623/src/LazyOperations/Intersection.jl#L723-L725)
```julia
function concretize(cap::Intersection)
    return intersection(concretize(cap.X), concretize(cap.Y))
end
```

### Struct-of-Arrays and Array-of-Structs

In [35]:
soa = (a=[1,2,3], b=[10, 20, 30])
aos = [(a=1, b=10), (a=2, b=20), (a=3, b=30)]

aos_eager = map(NamedTuple{(:a, :b)} ∘ tuple, soa.a, soa.b)

3-element Vector{NamedTuple{(:a, :b), Tuple{Int64, Int64}}}:
 (a = 1, b = 10)
 (a = 2, b = 20)
 (a = 3, b = 30)

In [36]:
soa_eager = NamedTuple{(:a, :b)}(tuple(getindex.(aos, :a), getindex.(aos, :b)))

(a = [1, 2, 3], b = [10, 20, 30])

In [37]:
@xquote map(NamedTuple{(:a, :b)} ∘ tuple, soa.a, soa.b)

Call(Some(map), FrankenTuple((Call(Some(∘), FrankenTuple((Some(NamedTuple{(:a, :b), T} where T<:Tuple), Some(tuple)), NamedTuple())), Some([1, 2, 3]), Some([10, 20, 30])), NamedTuple()))

September 2020:
The composition operator `∘` now returns a [`Base.ComposedFunction`](https://github.com/jw3126/julia/blob/37908d66492f3cf95759f8e0e7e3d2bd6d038c0c/base/operators.jl#L909-L914) instead of an anonymous function ([#37517]).

```julia
struct ComposedFunction{O,I} <: Function
    outer::O
    inner::I
end
```

## Different niche from Computer Algebra System

[SymbolicUtils.Term](https://github.com/JuliaSymbolics/SymbolicUtils.jl/blob/c3081cdbf59b8e07ed3a757d8e9eb8bdbc9cad6e/src/types.jl#L323-L328) intentionally does not dispatch on function or argument types:

```julia
struct Term{T, M} <: Symbolic{T}
    f::Any
    arguments::Any
    metadata::M
    hash::Ref{UInt} # hash cache
end
```


## Taking the idea too far?

In [38]:
typeof(im)

Complex{Bool}

`im` is defined in terms of `Complex`. Going the other way

In [39]:
𝗶 = @xquote sqrt((-1)::::S)

Call(Some(sqrt), FrankenTuple((Val{-1}(),), NamedTuple()))

In [40]:
xeval(𝗶)

LoadError: DomainError with -1.0:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [41]:
c = @xquote 1 + 2*𝗶

Call(Some(+), FrankenTuple((Some(1), Call(Some(*), FrankenTuple((Some(2), Some(Call(Some(sqrt), FrankenTuple((Val{-1}(),), NamedTuple())))), NamedTuple()))), NamedTuple()))

In [42]:
reinterpret(Int, [c])

2-element reinterpret(Int64, ::Vector{FixArgs.Call{Some{typeof(+)}, FrankenTuples.FrankenTuple{Tuple{Some{Int64}, FixArgs.Call{Some{typeof(*)}, FrankenTuples.FrankenTuple{Tuple{Some{Int64}, Some{FixArgs.Call{Some{typeof(sqrt)}, FrankenTuples.FrankenTuple{Tuple{Val{-1}}, (), Tuple{}}}}}, (), Tuple{}}}}, (), Tuple{}}}}):
 1
 2

Note that
- `@xquote 1 + 2*𝗶`
- `@xquote 2*𝗶 + 1`
- `@xquote 𝗶*2 + 1`
are all different types.

## Downsides
- code reuse increases coupling
- burden on the compiler
- huge types
- method ambiguities

# Thanks!